# Lab 5: Human in the Loop


## セットアップと初期化

環境のセットアップと必要なモジュールのインポートから始めます。重要なコンポーネントはチェックポインターです：

```python
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
```

チェックポインターはGitのような分散バージョン管理システムと同様に機能しますが、AI状態用です。AIの意思決定プロセスの異なる時点で状態の「コミット」を作成し、必要に応じて「ブランチ」または「リバート」することができます。

大きな状態オブジェクトを扱う場合、チェックポイントの過剰な使用は大量のメモリ使用につながる可能性があることに注意してください。長時間実行されるアプリケーションでは、古いチェックポイントを管理または期限切れにする戦略を実装してください。それらのチェックポイントが不要になった場合、データベースにTime-To-Liveを実装することを検討してください。

チェックポイント用のデータベースとしてSQLiteを使用しますが、本番アプリケーションではRedisまたはPostgresに切り替えることをお勧めします。

In [1]:
from dotenv import load_dotenv
import json
import os
import re
import sys
import warnings
import boto3
from botocore.config import Config
warnings.filterwarnings("ignore")
import logging

# ローカルモジュールのインポート
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# 基本設定
logger = utils.set_logger()  # ロガーの設定
pp = utils.set_pretty_printer()  # 整形出力用のプリンターを設定

# .envファイルまたはSecret Managerから環境変数を読み込む
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")  # AWS地域を環境変数から取得
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)  # Tavily APIキーを取得

# Bedrockの設定
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={"max_attempts": 0})  # タイムアウトと再試行の設定

# Bedrockランタイムクライアントの作成
bedrock_rt = boto3.client("bedrock-runtime", region_name=aws_region, config=bedrock_config)

# 利用可能なモデルを確認するためのBedrockクライアントの作成
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)

[2025-07-10 04:14:12,397] p2900569 {utils.py:66} INFO - TAVILY_API_KEY variable correctly retrieved from the .env file.


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_aws import ChatBedrockConverse
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

次に、カスタムメッセージ処理機能を備えたエージェント状態をセットアップします：

In [3]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
以前の例では、`messages`状態キーにデフォルトの`operator.add`または`+`リデューサーを
注釈付けしていました。これは常に新しいメッセージを既存のメッセージ配列の末尾に
追加します。

今回は、既存のメッセージを置き換えるサポートをするために、`messages`キーに
カスタムリデューサー関数で注釈を付けます。これにより、同じ`id`を持つメッセージを
置き換え、そうでない場合は追加します。
"""

# この関数は、メッセージの一貫性を保つための「スマートなメッセージブローカー」として機能します。同じIDを持つメッセージを更新し、新しいメッセージを追加します。
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # idを持たないメッセージにidを割り当てる
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    
    # 新しいメッセージを既存のメッセージとマージする
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # 同じidを持つ既存のメッセージを置き換える
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # 新しいメッセージを末尾に追加する
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

この関数は、マイクロサービスアーキテクチャにおけるスマートなメッセージブローカーのように機能します。会話履歴の一貫性と最新性を確保し、長時間実行されるAIインタラクションでコンテキストを維持するために重要です。

## ツールとエージェントのセットアップ

Tavilyサーチツールを統合します：

In [4]:
tool = TavilySearchResults(max_results=2)

/tmp/ipykernel_2900569/4289725543.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


## エージェントクラスにおける人間の介入

`Agent`クラスは、AIの意思決定プロセスにおける人間の介入を可能にするいくつかの重要な機能を実装しています：

1. **アクション前の中断**：
```python
self.graph = graph.compile(checkpointer=checkpointer, interrupt_before=["action"])
```

この行は「action」ノードの前に中断を設定します。これにより、アクションが実行される前に人間による監視が可能になり、レビューや修正の機会が提供されます。 次のラボで見るように、必要なノードの前に中断を追加することができます。

状態の検査： `exists_action`メソッドは現在の状態を出力します：

```python
def exists_action(self, state: AgentState):
    print(state)
    # ...
```

これにより、人間はAIの現在の状態（推論や意図されたアクションを含む）を検査することができます。

アクションの可視性： `take_action`メソッドでは、各ツール呼び出しが出力されます：

```python
print(f"Calling: {t}")
```

これにより、AIが実行しようとしているアクションの可視性が提供され、潜在的な人間の介入が可能になります。

チェックポイント： コンストラクタの`checkpointer`パラメータは、状態の保存と読み込みを可能にします。これにより「タイムトラベル」機能が実現され、人間は以前の決定ポイントを再訪し、潜在的に修正することができます。

変更可能な状態： `AgentState`は変更可能な構造です。このクラスで直接示されていませんが、状態の修正を可能にし、人間がAIのコンテキストや決定を変更できるようにします。

これらの機能が集合的に、人間がAIの意思決定プロセスを重要な節目で監視、介入、誘導できる人間参加型AIのフレームワークを作成します。

In [5]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        # システムメッセージを設定
        self.system = system
        # 状態グラフを初期化
        graph = StateGraph(AgentState)
        # LLMノードを追加（Bedrockを呼び出す）
        graph.add_node("llm", self.call_bedrock)
        # アクションノードを追加（ツールを実行する）
        graph.add_node("action", self.take_action)
        # 条件付きエッジを追加：ツール呼び出しがあればアクションへ、なければ終了
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        # アクションからLLMへのエッジを追加（ツール実行後に再びLLMへ）
        graph.add_edge("action", "llm")
        # エントリーポイントをLLMに設定
        graph.set_entry_point("llm")
        # チェックポインターを使用してグラフをコンパイル、アクションノード前に中断を設定
        self.graph = graph.compile(checkpointer=checkpointer, interrupt_before=["action"])
        # ツールを名前でアクセスできるように辞書に格納
        self.tools = {t.name: t for t in tools}
        # モデルにツールをバインド
        self.model = model.bind_tools(tools)

    def call_bedrock(self, state: AgentState):
        # 現在のメッセージ履歴を取得
        messages = state["messages"]
        # システムメッセージがあれば先頭に追加
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        # モデルを呼び出して応答を取得
        message = self.model.invoke(messages)
        # 応答をメッセージリストに追加して返す
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        # デバッグ用に現在の状態を出力
        print(state)
        # 最新のメッセージを取得
        result = state["messages"][-1]
        # ツール呼び出しがあるかどうかを確認
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        # 最新のメッセージからツール呼び出しを取得
        tool_calls = state["messages"][-1].tool_calls
        results = []
        # 各ツール呼び出しを実行
        for t in tool_calls:
            # 実行するツール呼び出しを出力（デバッグ用）
            print(f"Calling: {t}")
            # ツールを名前で呼び出し、引数を渡して実行
            result = self.tools[t["name"]].invoke(t["args"])
            # 結果をツールメッセージとして追加
            results.append(ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result)))
        # デバッグ用メッセージを出力
        print("Back to the model!")
        # ツール実行結果をメッセージリストとして返す
        return {"messages": results}

`interrupt_before=["action"]`パラメータは、AI パイプラインにおける重要な制御ポイントを実装します。

これはCI/CDパイプラインの承認ゲートの実装に類似しており、必要なチェックなしに重要なアクションが実行されないことを保証します。

AI安全性と制御メカニズムの理解を深めるために、2016年のGoogleやStanford、Berkeleyの研究者による論文["Concrete Problems in AI Safety"](https://arxiv.org/abs/1606.06565)を探索してみてください。

## エージェントの初期化と実行

Amazon Bedrock上のClaudeでエージェントを初期化します：

In [6]:
prompt = """あなたはスマートな研究アシスタントです。検索エンジンを使用して情報を調べてください。\
複数の呼び出しを行うことができます（同時または連続して）。\
何を求めているか確信がある場合にのみ、情報を検索してください。\
フォローアップの質問をする前に情報を調べる必要がある場合は、それも許可されています！"""

# Amazon Bedrockの Claude 3 Haikuモデルを初期化
model = ChatBedrockConverse(
    client=bedrock_rt,
    model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0,  # 決定論的な応答のために温度を0に設定
    max_tokens=None,  # トークン制限なし
)

# エージェントを初期化（モデル、ツール、システムプロンプト、チェックポインターを設定）
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [7]:
# ベルリンの天気について質問するメッセージを作成
messages = [HumanMessage(content="ベルリンの天気はどうですか？")]

# スレッドIDを設定（会話の永続性を管理するため）
thread = {"configurable": {"thread_id": "1"}}

# エージェントのグラフを実行し、イベントをストリーミング
for event in abot.graph.stream({"messages": messages}, thread):
    # 各イベントの値を出力
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='ベルリンの天気はどうですか？', additional_kwargs={}, response_metadata={}, id='7be52f3e-9e5c-42df-ad5b-f4f5eba0f4e3'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8CIf9WwymuABg'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '5c1fc527-58bd-4cfa-8039-c70674e2b712', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:14:50 GMT', 'content-type': 'application/json', 'content-length': '316', 'connection': 'keep-alive', 'x-amzn-requestid': '5c1fc527-58bd-4cfa-8039-c70674e2b712'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1078]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--fb1bd67f-4559-42e4-92a0-a3b52fc0d58d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8CIf9WwymuABg', 'type':

In [8]:
thread

{'configurable': {'thread_id': '1'}}

## 状態の検査と修正

エージェントの状態を検査および修正する能力は強力な機能です：

In [9]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='ベルリンの天気はどうですか？', additional_kwargs={}, response_metadata={}, id='7be52f3e-9e5c-42df-ad5b-f4f5eba0f4e3'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8CIf9WwymuABg'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '5c1fc527-58bd-4cfa-8039-c70674e2b712', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:14:50 GMT', 'content-type': 'application/json', 'content-length': '316', 'connection': 'keep-alive', 'x-amzn-requestid': '5c1fc527-58bd-4cfa-8039-c70674e2b712'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1078]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--fb1bd67f-4559-42e4-92a0-a3b52fc0d58d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8C

この機能は、本番環境でのライブデバッガーを持つことに似ており、実行中のプロセスの状態を検査および修正することができます。

In [10]:
abot.graph.get_state(thread).next

('action',)

ご覧のように、次に実行されるノードは`('action',)`ノードです。

私たちは`action`ノードの前に中断を設定してグラフをコンパイルしたことを覚えておいてください。

```python
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
```


### ...中断後に続行

アクション（tavily-aiの呼び出し）の前に停止したので、続行することができます。それがどのように機能するか見てみましょう。

In [11]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8CIf9WwymuABg', 'type': 'tool_call'}


Back to the model!
{'messages': [ToolMessage(content="[{'title': 'Berlin weather in July 2025 - Weather25.com', 'url': 'https://www.weather25.com/europe/germany/berlin?page=month&month=July', 'content': '| June | 23° / 13° | 5 | 25 | 0 | 96 mm | Perfect | Berlin in June |\\n| July | 25° / 15° | 6 | 25 | 0 | 122 mm | Perfect | Berlin in July |\\n| August | 25° / 15° | 4 | 27 | 0 | 87 mm | Perfect | Berlin in August |\\n| September | 20° / 12° | 4 | 26 | 0 | 55 mm | Good | Berlin in September |\\n| October | 14° / 7° | 5 | 25 | 0 | 66 mm | Good | Berlin in October |\\n| November | 8° / 4° | 6 | 22 | 2 | 60 mm | Bad | Berlin in November | [...] | Month | Temperatures | Rainy Days | Dry Days | Snowy Days | Rainfall | Weather | More details |\\n| --- | --- | --- | --- | --- | --- | --- | --- |\\n| January | 3° / -2° | 8 | 11 | 12 | 72 mm | Awful | Berlin in January |\\n| February | 4° / -1° | 6 | 13 | 9 | 50 mm | Awful | Berlin in February |\\n| March | 9° / 2° | 6 | 20 | 5 | 51 mm | Bad | 

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='ベルリンの天気はどうですか？', additional_kwargs={}, response_metadata={}, id='7be52f3e-9e5c-42df-ad5b-f4f5eba0f4e3'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8CIf9WwymuABg'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '5c1fc527-58bd-4cfa-8039-c70674e2b712', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:14:50 GMT', 'content-type': 'application/json', 'content-length': '316', 'connection': 'keep-alive', 'x-amzn-requestid': '5c1fc527-58bd-4cfa-8039-c70674e2b712'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1078]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--fb1bd67f-4559-42e4-92a0-a3b52fc0d58d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Berlin current weather today'}, 'id': 'tooluse_FcYc0sd8T8C

In [13]:
abot.graph.get_state(thread).next

('action',)

### 人間の入力を追加する

次に、Webを検索する`action`の実行に対するあなたの承認を追加する方法を見ていきます。

JupyterLabで実行しているか、例えばVS Codeで実行しているかによって、入力ボックスはコードセルの下、または上部に表示されるはずです。

![ユーザー入力を求める入力ボックスの例](../assets/lab5_1.png)

検索される内容に問題がなければ、「y」（はい）を入力してください。それ以外の場合は、操作が中止されます。

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# ロサンゼルスの天気について質問するメッセージを作成
messages = [HumanMessage("ロサンゼルスの天気はどうですか？")]

# 新しいスレッドIDを設定
thread = {"configurable": {"thread_id": "2"}}

# 最初のイベントをストリーミング
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

# 状態を確認
if abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread), "\n")
    
    # ボタンを使用して承認を得る
    button = widgets.Button(description="続行する")
    display(button)
    
    def on_button_clicked(b):
        # ボタンがクリックされたら残りのイベントをストリーミング
        for event in abot.graph.stream(None, thread):
            for v in event.values():
                print(v)
    
    button.on_click(on_button_clicked)

{'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか？', additional_kwargs={}, response_metadata={}, id='1b7c719f-b261-43ba-845b-9f684fb59722'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_gvVJ9cgLQl-r10LAzl4GCQ'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'd8241ba4-ccbb-40f5-9e29-a095ff1bac92', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:16:10 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'd8241ba4-ccbb-40f5-9e29-a095ff1bac92'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1261]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--7fd0a8c9-6dac-4272-a811-f6dfd1d2cdaa-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_gvVJ9cgLQl-r10

Button(description='続行する', style=ButtonStyle())

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_gvVJ9cgLQl-r10LAzl4GCQ', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'title': 'Weather Forecast and Conditions for Los Angeles, CA', 'url': 'https://weather.com/weather/today/l/Los+Angeles+CA?canonicalCityId=7d3c65d8b80674fb48647ddbc936bb8b', 'content': 'Los Angeles, CA. As of 6:12 pm PDT. 77°. Sunny. Day 85° • Night 63°. Air Quality Alert. Weather Today in Los Angeles, CA. Feels Like77°. 5:49 am. 8:07 pm. High', 'score': 0.55420566}, {'title': 'Los Angeles weather in July 2025 | Weather25.com', 'url': 'https://www.weather25.com/north-america/usa/california/los-angeles?page=month&month=July', 'content': '| May | 25° / 15° | 0 | 31 | 0 | 4 mm | Perfect | Los Angeles in May |\\n| June | 27° / 17° | 0 | 30 | 0 | 1 mm | Perfect | Los Angeles in June |\\n| July | 30° / 20° | 0 | 31 | 0 | 5 mm | Good | Los Angeles in July |\\n| August | 31

次に実行されるノードは `('action',)` ノードです。覚えておいてください、私たちは `action` ノードの前に割り込みを設定してグラフをコンパイルしました。

```python
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
```

---


非常に興味深いことの一つは、すべてのツールではなく、特定のツールセットを呼び出す際にのみ停止して人間の入力を求めることです。続きを読む前に、それがどのように実現できるかを考えてみてください。

### 特定のツール呼び出しでのみ停止する

1. ツール呼び出しを解析し、ツール呼び出しパラメータ `name`（ツール名）が停止パラメータと同じ場合にのみ実行を停止する。
2. 停止が必要なすべてのツールを追加のノードに追加する。

一般的に、オプション2の方がよりクリーンでデバッグしやすいです。これがより大きな例でどのように実装されているかを見たい場合は、LangGraphの例から[カスタマーサポートエージェント](https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/#part-3-conditional-interrupt)を参照して、機密性の高いツールがどのように処理されているかを確認してください。

グラフのプレビューはこちらです：
![機密ツールと安全なツールを持つカスタマーサポートエージェントのグラフ](../assets/lab5_2.png)

## Modify State

割り込みまでStateRunを変更し、その後状態を変更する。

In [15]:
messages = [HumanMessage("ロサンゼルスの天気はどうですか?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw

In [16]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 't

In [17]:
current_values = abot.graph.get_state(thread)

In [18]:
current_values.values["messages"][-1]

AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A', 'type': 'tool_call'}], usage_metadata={'input_tokens': 501, 'output_tokens': 62, 'total_tokens': 563, 'input_token_details': {'cache

In [19]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'Los Angeles current weather forecast'},
  'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A',
  'type': 'tool_call'}]

In [ ]:
# このコードは、エージェントの状態（この場合は検索クエリ）を変更する方法を示しています。
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "東京の現在の天気は"},
        "id": _id,
    }
]

In [23]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '東京の現在の天気は'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], usage_metadata

{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f05d451-de1b-6cd2-8003-e42deaccce59'}}

In [24]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '東京の現在の天気は'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS

In [25]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': '東京の現在の天気は'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A', 'type': 'tool_call'}


Back to the model!
{'messages': [ToolMessage(content="[{'title': '今日・明日と14日間(2週間)の1時間ごとの天気予報 - Toshin.com - 東進', 'url': 'https://www.toshin.com/weather/detail?id=66124', 'content': '- 東進ビジネススクール  \\n  大学生向け東進学力POS\\n\\n- 社会人対象\\n\\n- 東進ビジネススクール  \\n  社会人向け東進学力POS\\n\\nマイページ\\n\\n資料請求\\n\\n入学受付\\n\\n全国学校のお天気\\n\\n新規会員登録 ログイン 天気メールを設定する\\n\\n \\n\\nHome   >  東京都   >  千代田区   >  東京   >  14日間(2週間)の1時間ごとの天気予報\\n\\n   \\n\\n雨雲レーダー\\n\\n \\n\\n# 東京の天気\\n\\n6月5日 13:00発表\\n\\nリロードすると1時間ごとに更新されます。\\n\\n今日\\n\\n2025年\\n\\n6月5日(木)\\n\\n晴れ\\n\\n28℃/19℃\\n\\n降水確率\\n\\n0%\\n\\n明日\\n\\n2025年\\n\\n6月6日(金)\\n\\n晴れ\\n\\n28℃/18℃\\n\\n降水確率\\n\\n0%\\n\\n明日\\n\\n2025年\\n\\n6月6日(金)\\n\\n晴れ\\n\\n28℃/18℃\\n\\n降水確率\\n\\n0%\\n\\n2025年\\n\\n6月7日(土)\\n\\n晴れ時々曇\\n\\n26℃/19℃\\n\\n降水確率\\n\\n20%\\n\\n2025年\\n\\n6月7日(土)\\n\\n晴れ時々曇\\n\\n26℃/19℃\\n\\n降水確率\\n\\n20%\\n\\n2025年\\n\\n6月8日(日)\\n\\n時々曇り\\n\\n29℃/21℃\\n\\n降水確率 [...] 最低\\n\\n21℃\\n\\n降水量\\n\\n0.0mm\\n\\n湿度\\n\\n73%\\n\\n風速\\n\\n3m/s\\n\\n風向\\n\\n南\\n\\n最高\\n\\n28℃

## Time Travel


In [26]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '東京の現在の天気は'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS

同じ状態を取得するために、以下のオフセットを `-1` から `-3` に変更します。これは、ソフトウェアの最新バージョンで状態メモリに保存されるようになった初期状態 `__start__` と最初の状態を考慮したものです。

In [ ]:
# 過去の状態を取得し、そこから再開することができます
to_replay = states[-3]

In [28]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 't

In [29]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather Forecast and Conditions for Los Angeles, CA\', \'url\': \'https://weather.com/weather/today/l/Los+Angeles+CA?canonicalCityId=7d3c65d8b80674fb48647ddbc936bb8b\', \'content\': "© The Weather Company, LLC 2025 [...] ## recents\\n\\nMenu\\n\\n## Weather Forecasts\\n\\n## Radar & Maps\\n\\n## News & Media\\n\\n## Products & Account\\n\\n## Lifestyle\\n\\n### Specialty Forecasts\\n\\n# Los Angeles, CA\\n\\n## Air Quality Alert\\n\\n## Weather Today in Los Angeles, CA\\n\\n5:49 am\\n\\n8:07 pm\\n\\n### Morning\\n\\n### Afternoon\\n\\n### Evening\\n\\n### Overnight\\n\\n## Don\'t Miss\\n\\n## Hourly Forecast\\n\\n### Now\\n\\n### 10 pm\\n\\n### 11 pm\\n\\n### 12 am\\n\\n### 1 am\\n\\n## Seasonal Hub\\n\\n## Trending Now\\n\\n## Daily Forecast\\n\\n### T

## Go back in time and edit


In [30]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 't

In [31]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]
to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "現在の東京の天気は？, accuweather"},
        "id": _id,
    }
]

In [33]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '現在の東京の天気は？, accuweather'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], 

In [34]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': '現在の東京の天気は？, accuweather'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Tokyo, Tokyo, Japan Weather Forecast - AccuWeather\', \'url\': \'https://www.accuweather.com/en/jp/tokyo/226396/weather-forecast/226396\', \'content\': \'9 AM Image 689° Image 7: rain drop6%10 AM Image 890° Image 9: rain drop5%11 AM Image 1092° Image 11: rain drop5%12 PM Image 1293° Image 13: rain drop8%1 PM Image 1495° Image 15: rain drop44%2 PM Image 1694° Image 17: rain drop45%3 PM Image 1893° Image 19: rain drop51%4 PM Image 2092° Image 21: rain drop18%5 PM Image 2291° Image 23: rain drop14%6 PM Image 2489° Image 25: rain drop10%7 PM Image 2687° Image 27: rain drop2%8 PM Image 2886° Image 29: rain drop2% [...] © 2025 AccuWeather, Inc. "AccuWeather" and sun design are registered trademarks of AccuWeather, Inc. All Rights Reserved. \\n\\nTerms of Use | Privacy Pol

## Add message to a state at a given time


In [35]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '現在の東京の天気は？, accuweather'}, 'id': 'tooluse_nG_gPP

In [36]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]

In [37]:
# 湿度を不可能な値に更新しましょう
state_update = {"messages": [ToolMessage(tool_call_id=_id,name="tavily_search_results_json",content="23度、湿度110パーセント",)]}

In [38]:
branch_and_add = abot.graph.update_state(
    to_replay.config, state_update, as_node="action"
)

In [39]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='ロサンゼルスの天気はどうですか?', additional_kwargs={}, response_metadata={}, id='780ff10b-94ab-405c-afc0-e0c305b8a1ed'), AIMessage(content=[{'type': 'tool_use', 'name': 'tavily_search_results_json', 'input': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw8C83zS_A'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'b033167c-708f-4a8c-b92d-a644578cb5fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 10 Jul 2025 04:17:48 GMT', 'content-type': 'application/json', 'content-length': '434', 'connection': 'keep-alive', 'x-amzn-requestid': 'b033167c-708f-4a8c-b92d-a644578cb5fb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1213]}, 'model_name': 'us.anthropic.claude-3-5-haiku-20241022-v1:0'}, id='run--bea26819-7290-44d1-a6a2-dd50b42cac61-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles current weather forecast'}, 'id': 'tooluse_nG_gPPunQ3ytjw